In [108]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
from brainviewer import NapariBrainViewer

# Working With Point Clouds

this section show examples of how to work with point clouds of neurons and there respective activities.

## Generating very very bad fake neural activity

In [110]:
import numpy as np
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt

In [111]:
# parameters of the simulation
space_size = 100
s_xyz = np.array([space_size,space_size,space_size])

n_regions = 15
n_times = 3000
n_neurons = 5000

In [112]:
# creating regions

## spatial
com_regions = np.random.uniform(low=0*s_xyz, high=s_xyz, size=(n_regions, 3)) # centers of masse


## temporal
p_join_regions = np.random.choice( # probabilities of 2 regions activating simultaneously
    [0,1,0.1,0.9,0],
    p = [0.3,0.3,0.175,0.175,0.05],
    size=(n_regions, n_regions),
)
p_join_regions = ( p_join_regions + p_join_regions.conj().T )/2

region_acts = np.zeros((n_times, n_regions)) # random telegraph signal for each region with p_join contraint
p = 0.001
ref_j = 0
for i in range(1,n_times):
    if np.random.rand() < p*2:
        ref_j = np.random.randint(0,n_regions)
    if np.random.rand() < p:
        region_acts[i:, ref_j] = 1-region_acts[i-1, ref_j]
    for j in range(n_regions):
        if j == ref_j:
            continue
        if np.random.rand() < p_join_regions[ref_j,j]:
            region_acts[i:,j] = region_acts[i:,ref_j]
        else:
            region_acts[i:,j] = 1-region_acts[i:,ref_j]

In [113]:
# creating neurons

## spatial
com_neurons = np.random.uniform(low=0*s_xyz, high=s_xyz, size=(n_neurons, 3)) # centers of masse


## participation of neurons in each region
dists = distance_matrix(com_neurons, com_regions)
def norm_tanh(x, x0=20, width=10):
    xmin, xmax = x.min(), x.max()
    x = (x-xmin) / (xmax - xmin)
    x0 = (x0-xmin) / (xmax - xmin)
    width = (width-xmin) / (xmax - xmin)
    y = (1 - np.tanh((x-x0)/width*np.pi))/2
    return y/y.max()
participation_factor = norm_tanh(dists)

## temporal
neuron_tendencies = region_acts@participation_factor.T
neuron_spikes = np.random.poisson(neuron_tendencies*5)

## Opening the brain viewer

In [114]:
nbv = NapariBrainViewer()

## Scatter Neurons

In [115]:
layer_com = nbv.points(com_neurons, blending="translucent_no_depth", size=2)

## Scatter neurons with single frame activity

In [116]:
from brainviewer import cm_inferno_alpha

In [117]:
layer_activity = nbv.points(
    com_neurons, neuron_spikes[50,:], 
    cmap=cm_inferno_alpha, crange=(0,+5), 
    blending="translucent_no_depth",
    size=2,
)

## Display neuron activity as interactive line plot

In [118]:
from brainviewer import ActivityViewer, PointLayerSelector

In [119]:
av = ActivityViewer(nbv)
pls = PointLayerSelector(layer_com, av, neuron_spikes)

INFO: Right Click on a neuron to display its activity.


## Display interactive correlation maps 

In [120]:
from brainviewer import PointLayerPairwise
from brainviewer import cm_seismic_alpha

In [121]:
C = np.corrcoef(neuron_spikes.T) # correlations between neurons
C[~np.isfinite(C)] = 0

/home/ljp/.local/bin/miniforge3/envs/brainviewer-env/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/ljp/.local/bin/miniforge3/envs/brainviewer-env/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [122]:
plp = PointLayerPairwise(layer_com, C, cm_seismic_alpha, (-1,+1))

INFO: Right Click on a neuron to display Pairwise.


In [134]:
mat=np.random.random((5,5))
reg = np.random.randint(0,5,3)

mat[:,reg] = [0,1,0]

mat

array([[1.        , 0.08125495, 0.        , 0.83270715, 0.        ],
       [1.        , 0.97289868, 0.        , 0.5808309 , 0.        ],
       [1.        , 0.21158132, 0.        , 0.89195438, 0.        ],
       [1.        , 0.30064228, 0.        , 0.52300803, 0.        ],
       [1.        , 0.74073785, 0.        , 0.76919376, 0.        ]])